# CS404 ASSIGNMENT 1: SOLVING COLOR-MAZE PUZZLE SEARCH

Berkay Barış Turan - 28132

This notbook consists of the implementation of the following algorithms to solve the provided version of the Color-Maze puzzle:
* Uniform Cost Search (UCS) algorithm
* A* Search algorithm

In [ ]:
import itertools
from queue import PriorityQueue
from copy import deepcopy
import psutil
import os

### Node Class Definition

In [ ]:
class node:

    id_iter = itertools.count(1) # for assigning id number to every node automatically

    def __init__(self, parent_id= None, cost=0, state=[], action_seq=[]):
        self.id = next(self.id_iter)
        self.cost = cost
        self.parent_id = parent_id
        self.state = state
        self.action_seq = action_seq


    def __str__(self):
      action_seq_string = ""
      for action in self.action_seq:
        action_seq_string += " " + action

      return "Node " + str(self.id) + " | Parent Node: " + str(self.parent_id) +" | Cost: " + str(self.cost) +" | Action Sequence: " + action_seq_string

    def __gt__(self,other):
      colored_state=0
      for i in range (len(self.state)):
        for j in range(len(self.state[0])):
          if self.state[i][j] == "C":
            colored_state += 1

      colored_state_other=0
      for i in range (len(other.state)):
        for j in range(len(other.state[0])):
          if other.state[i][j] == "C":
            colored_state_other += 1

      if(self.id > other.id):
        return False
      else:
        return True

### Helper Functions

In [ ]:
def printBoard(GameBoard):
# for printing the state
  for i in range(len(GameBoard)):
    for j in range(len(GameBoard[0])):
      print(GameBoard[i][j], end=" ")
    print(" ")


def find_loc_agent(state):
  # returns row_id, col_id that the agent is located
  for i in range(row):
    for j in range(col):
      if state[i][j] == "S":
        return i, j


def actions(state):
  # returns a list of possible actions that an agent can perform

  #id_act = itertools.count()

  agent_r, agent_c = find_loc_agent(state)
  possible_actions = deepcopy(action_directions)

  # eliminating direction L (Left)
  if agent_c == 0:
    possible_actions.remove("L")

  elif state[agent_r][agent_c-1] == "X":
    possible_actions.remove("L")

  # eliminating direction U (Up)
  if agent_r == 0:
    possible_actions.remove("U")

  elif state[agent_r-1][agent_c] == "X":
    possible_actions.remove("U")

  # eliminating direction R (Right)
  if agent_c == (col-1):
    possible_actions.remove("R")

  elif state[agent_r][agent_c+1] == "X":
    possible_actions.remove("R")

  #eliminating direction D (Down)
  if agent_r == (row-1):
    possible_actions.remove("D")

  elif state[agent_r+1][agent_c] == "X":
    possible_actions.remove("D")

  #print("Function: actions() | Iteration: ", next(id_act), " | Possible Actions: ", possible_actions)
  return possible_actions


def new_state_cost(stat, action):
  # retuns the new state as a result of a given action and its cost

  #id_nsc = itertools.count()
  state = deepcopy(stat)
  agent_r, agent_c = find_loc_agent(state)

  if action == "U":
    curr_r = agent_r
    while(curr_r > 0 and state[curr_r-1][agent_c] != "X"):
      state[curr_r-1][agent_c] = "S"
      state[curr_r][agent_c] = "C"
      curr_r -= 1
    cost = abs(agent_r - curr_r)
    #print("Function: new_state_cost() | Iteration: ", next(id_nsc), " | Action: ", action ," | Cost: ", cost, " | State: ", printBoard(state) )
    return cost, state

  elif action == "D":
    curr_r = agent_r
    while(curr_r < (row-1) and state[curr_r+1][agent_c] != "X"):
      state[curr_r+1][agent_c] = "S"
      state[curr_r][agent_c] = "C"
      curr_r += 1
    cost = abs(agent_r - curr_r)
    #print("Function: new_state_cost() | Iteration: ", next(id_nsc), " | Action: ", action ," | Cost: ", cost, " | State: ", printBoard(state) )
    return cost, state

  elif action == "L":
    curr_c = agent_c
    while(curr_c > 0 and state[agent_r][curr_c-1] != "X"):
      state[agent_r][curr_c-1] = "S"
      state[agent_r][curr_c] = "C"
      curr_c -= 1
    cost = abs(agent_c - curr_c)
    #print("Function: new_state_cost() | Iteration: ", next(id_nsc), " | Action: ", action ," | Cost: ", cost, " | State: ", printBoard(state) )
    return cost, state

  elif action == "R":
    curr_c = agent_c
    while(curr_c < (col-1) and state[agent_r][curr_c+1] != "X"):
      state[agent_r][curr_c+1] = "S"
      state[agent_r][curr_c] = "C"
      curr_c += 1
    cost = abs(agent_c - curr_c)
    #print("Function: new_state_cost() | Iteration: ", next(id_nsc), " | Action: ", action ," | Cost: ", cost, " | State: ", printBoard(state) )
    return cost, state


def successors(stat, current_node):
    # retuns a list of possible successor nodes/states
    succ = []
    #print("Current Parent Node: ", current_node)

    possible_actions = actions(stat)

    for action in possible_actions:
      #id_adc = itertools.count()
      state = deepcopy(stat)
      cost, st = new_state_cost(stat, action)
      actions_l = deepcopy(current_node.action_seq)
      actions_l.append(action)
      n = node(cost=(current_node.cost+cost), parent_id=current_node.id, state=st, action_seq= actions_l)
      succ.append((n.cost, n))
      #print("Function: successors()-for loop | Iteration: ", next(id_adc), " | Possible Actions: ", possible_actions," | Action: ", action, " | Actions List: ", actions_l, " | State: ", printBoard(state))
      #print("--------------------------------------------------------------------------------------")

    #for c, n in succ:
    #  print(c, n.id, n.parent_id ,n.action_seq,)

    return succ


def successors_A_star(stat, current_node):
    succ = []
    #print("Current Parent Node: ", current_node)

    possible_actions = actions(stat)

    for action in possible_actions:
      #id_adc = itertools.count()
      state = deepcopy(stat)
      cost, st = new_state_cost(stat, action)
      actions_l = deepcopy(current_node.action_seq)
      actions_l.append(action)

      if (current_node.cost+cost + heuristic(st)) <= optimal_fn:
        n = node(cost=(current_node.cost+cost), parent_id=current_node.id, state=st, action_seq= actions_l)
        succ.append((fn(n), n))
        #print("Function: successors()-for loop | Iteration: ", next(id_adc), " | Possible Actions: ", possible_actions," | Action: ", action, " | Actions List: ", actions_l, " | State: ", printBoard(state))
        #print("--------------------------------------------------------------------------------------")

    #for c, n in succ:
    #  print(c, n.id, n.parent_id ,n.action_seq,)

    return succ

def heuristic(state):
# calculates the number of invisited cells remained ("0") in that node's state
  zeros = 1
  for i in range(len(state)):
    for j in range(len(state[0])):
      if state[i][j] == "0":
        zeros += 1
  return zeros

def fn(node):
# calculates the f-value of a given node
  return (heuristic(node.state) + node.cost)


def is_goal(state):
  #goal function
  for i in range(len(state)):
    for j in range(len(state[0])):
      if state[i][j] == "0":
        return False
  return True

### Uniform Cost Search (UCS) Algorithm

In [ ]:
def uniform_cost_search(initial_state):
    #Uniform Cost Search Function
    node_list = []
    n = node(state = initial_state)
    frontier = PriorityQueue()
    frontier.put((n.cost, n))
    explored = []

    while not frontier.empty():
        (cost, current_node) = frontier.get()
        node_list.append(current_node)
        state = deepcopy(current_node.state)
        if is_goal(state):
            print("Uniform Cost Search Information \n")
            #for i in range(len(node_list)):
            #  print(node_list[i])
            print("\n Total Number of Nodes Explored: ",current_node.id ,"\n")
            return current_node
        if state not in explored:
            explored.append(state)
            for (succ_cost, succ) in successors(state, current_node):
                #print((succ_cost, succ.id))
                frontier.put((succ_cost, succ))
    return None

### A* Search Algorithm

In [ ]:
def A_star_search(initial_state, optimal_fn):
    # A* search function
    # optimal_fn --> If a nodes fn value acceeds the optimal one, the node is discarded
    node_list = []
    n = node(state = initial_state)
    frontier = PriorityQueue()
    frontier.put((fn(n), n))
    explored = []

    while not frontier.empty():
        (f_value, current_node) = frontier.get()
        node_list.append(current_node)
        state = deepcopy(current_node.state)
        if is_goal(state):
            print("A* Search Information \n")
            #for i in range(len(node_list)):
            #  print(node_list[i])
            print("\n Total Number of Nodes Explored: ",current_node.id ,"\n")
            return current_node
        if state not in explored:
            explored.append(state)
            for (fn_value, succ) in successors_A_star(state, current_node):
                #print((succ_cost, succ.id))
                if fn_value <= optimal_fn:
                  #print("fn_value:", fn_value, " | optimal_fn:" , optimal_fn)
                  frontier.put((fn_value, succ))
    return None

#### Test Cases

In [ ]:
# Basic Instances

basic1 =[["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "0", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "X", "0", "X", "0", "0", "0", "X", "0", "X"],
         ["X", "X", "0", "X", "X", "X", "0", "X", "0", "X"],
         ["X", "X", "0", "X", "X", "X", "0", "X", "0", "X"],
         ["X", "X", "0", "0", "0", "0", "0", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

basic2 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "0", "0", "0", "X", "X", "X", "X"],
         ["X", "X", "X", "0", "X", "0", "X", "X", "X", "X"],
         ["X", "X", "X", "0", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "0", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "0", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "0", "X", "0", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "0", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "X", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

basic3 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "0", "0", "0", "0", "0", "X", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "0", "X", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "0", "0", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["0", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["0", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "X", "X", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]


basic4 = [["0", "0", "0", "0", "X", "X", "X", "X", "X", "X"],
         ["0", "X", "X", "X", "0", "0", "0", "0", "X", "X"],
         ["0", "X", "X", "0", "0", "X", "X", "0", "X", "X"],
         ["0", "X", "X", "0", "X", "X", "X", "0", "0", "X"],
         ["0", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["0", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["0", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "X", "X", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

basic5 = [["X", "0", "0", "0", "0", "0", "0", "0", "X", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "0", "X", "X"],
         ["X", "X", "X", "0", "0", "0", "X", "0", "X", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "0", "0", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "X", "X", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]


normal1 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "X", "X", "0", "0", "0", "0", "0"],
         ["X", "X", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "0", "X"]]

normal2 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "X", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
         ["X", "0", "0", "0", "S", "0", "0", "0", "0", "X"],
         ["X", "0", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "0", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["0", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

normal3 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "0", "0", "0", "X", "X", "X", "0", "0", "X"],
         ["X", "X", "X", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "0", "0", "0", "0", "0", "0", "0", "X", "X"],
         ["X", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

normal4 = [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
         ["X", "0", "0", "0", "0", "0", "0", "0", "0", "0"],
         ["X", "0", "X", "X", "X", "0", "X", "X", "0", "0"],
         ["X", "0", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "0", "X", "X", "X", "0", "X", "X", "0", "X"],
         ["X", "0", "0", "0", "S", "0", "X", "X", "0", "X"],
         ["X", "0", "0", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "0", "X"],
         ["X", "X", "0", "0", "0", "0", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

normal5 = [["X", "0", "0", "0", "0", "0", "0", "0", "X", "X"],
         ["X", "X", "X", "X", "X", "0", "X", "0", "X", "X"],
         ["X", "X", "X", "0", "0", "0", "X", "0", "X", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "0", "0", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "X", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "0", "X", "0", "X", "X", "X", "X", "0", "X"],
         ["X", "S", "0", "0", "X", "X", "0", "0", "0", "X"],
         ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

difficult1 =  [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
              ["X", "X", "X", "X", "0", "0", "0", "0", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "0", "0", "0", "S", "0", "0", "0", "0", "0"],
              ["X", "0", "X", "X", "0", "X", "X", "X", "X", "X"],
              ["X", "0", "X", "X", "0", "X", "X", "X", "X", "X"],
              ["0", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
              ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

difficult2 =  [["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
              ["X", "X", "X", "X", "0", "0", "0", "0", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "X", "X", "X", "0", "X", "X", "X", "0", "X"],
              ["X", "0", "0", "0", "S", "0", "0", "0", "0", "0"],
              ["X", "0", "X", "X", "0", "X", "X", "X", "X", "X"],
              ["X", "0", "X", "X", "0", "X", "X", "X", "X", "X"],
              ["0", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
              ["X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]]

difficult3 =  [["X", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
              ["X", "0", "X", "X", "X", "X", "X", "X", "0", "X"],
              ["X", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
              ["X", "0", "0", "0", "X", "X", "0", "0", "0", "X"],
              ["X", "X", "X", "0", "X", "X", "0", "X", "X", "X"],
              ["X", "0", "0", "0", "X", "X", "0", "0", "0", "X"],
              ["X", "0", "0", "0", "X", "X", "0", "0", "0", "X"],
              ["X", "0", "X", "X", "X", "X", "X", "X", "0", "X"],
              ["X", "0", "0", "0", "0", "0", "0", "0", "0", "X"],
              ["X", "S", "0", "0", "X", "X", "0", "0", "0", "X"]]

#### Initialization of Initial State and Global Variables

In [ ]:
"""
maze = [["0", "0", "0", "0", "0", "0"],
        ["0", "X", "X", "X", "X", "0"],
        ["0", "0", "X", "X", "X", "0"],
        ["X", "X", "X", "X", "X", "0"],
        ["0", "X", "X", "X", "X", "0"],
        ["S", "0", "0", "0", "0", "0"]]
"""
maze= normal3

global row, col, action_directions

action_directions = ["L","R","U","D"]
row = len(maze)
col = len(maze[0])

print("Row Number of the maze: ", row)
print("Column Number of the maze: ", col)

Row Number of the maze:  10
Column Number of the maze:  10


#### Run Time of UCS

In [ ]:
solution_UCS = uniform_cost_search(maze)
print("Solution:", solution_UCS)
print("\n")
#printBoard(solution_UCS.state)

Actions = deepcopy(solution_UCS.action_seq)
state = deepcopy(maze)
for i in range(len(Actions)):
  cost, state = new_state_cost(state,Actions[i])
  print("Action:", Actions[i])
  printBoard(state)
  print("\n")
  state = state

print("\n")

# Calling psutil.cpu_precent() for 4 seconds
print('The CPU usage is: ', psutil.cpu_percent(4))
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

Uniform Cost Search Information 


 Total Number of Nodes Explored:  70 

Solution: Node 70 | Parent Node: 67 | Cost: 35 | Action Sequence:  R U L U R U R D L U L


Action: R
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X 0 0 0 0 0 0 0 0 X  
X X X X X X X X 0 X  
X C C C C C C C S X  
X X X X X X X X X X  


Action: U
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X 0 0 0 0 0 0 0 S X  
X X X X X X X X C X  
X C C C C C C C C X  
X X X X X X X X X X  


Action: L
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X S C C C C C C C X  
X X X X X X X X C X  
X C C C C C C C C X  
X X X X X X X X X X  


Action: U
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X S 0 0 0 0 0 0 X X

#### Run Time of A*

In [ ]:
optimal_fn = solution_UCS.cost + 1 # +1 is due to the fact that a heuristic function returns a positive number.

# Hardcoding the optimal solution
# In order to compare the CPU & RAM usages of the algorithms correctly
#optimal_fn = 21

node.id_iter = itertools.count(1)
solution_A_star = A_star_search(maze, optimal_fn)
print("Solution:", solution_A_star)
print("\n")

Actions = deepcopy(solution_A_star.action_seq)
state = deepcopy(maze)
for i in range(len(Actions)):
  cost, state = new_state_cost(state,Actions[i])
  print("Action:", Actions[i])
  printBoard(state)
  print("\n")
  state = state

print("\n")

# Calling psutil.cpu_precent() for 4 seconds
print('The CPU usage is: ', psutil.cpu_percent(4))
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

A* Search Information 


 Total Number of Nodes Explored:  16 

Solution: Node 16 | Parent Node: 15 | Cost: 35 | Action Sequence:  R U L U R U R D L U L


Action: R
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X 0 0 0 0 0 0 0 0 X  
X X X X X X X X 0 X  
X C C C C C C C S X  
X X X X X X X X X X  


Action: U
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X 0 0 0 0 0 0 0 S X  
X X X X X X X X C X  
X C C C C C C C C X  
X X X X X X X X X X  


Action: L
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X 0 0 0 0 0 0 0 X X  
X S C C C C C C C X  
X X X X X X X X C X  
X C C C C C C C C X  
X X X X X X X X X X  


Action: U
X X X X X X X X X X  
X X X X X X X X X X  
X X X X X X X X X X  
X 0 0 0 X X X 0 0 X  
X X X 0 0 0 0 0 0 X  
X S 0 0 0 0 0 0 X X  
X C C C